# F-тест: структурные изменения/сдвиги

In [ ]:
import pandas as pd
import numpy as np

from statsmodels.formula.api import ols
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования

from scipy.stats import f # f-распределение и критические значения

# Не показывать Warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

<i><b><span style="color: purple">Значимость структурных изменений/сдвигов</span> </b><br>

Пусть бинарная переменная $D$ отвечает за структурное изменение (сдвиги).

Для учёта структурных изменений (сдвигов) рассмотрим две модели

__(1) Модель без структурных изменений (сдвигов)__

$$
	(\log)y=\beta_0+\beta_1(\log)x_1+\cdots+\beta_k(\log)x_k+u=x^\top \beta+u
$$

Обозначим коэффициент детерминации $R^2$ для этой модели через $R^2_{res}$.

__(2) Модель со структурными изменениями (сдвигами)__

$$
	(\log)y=x^\top \beta+(D\cdot x^\top) \gamma+v
$$

Обозначим коэффициент детерминации $R^2$ для этой модели через $R^2_{unres}$.

Для регрессии (2)  тестируем гипотезу, что $J=k+1$ новых коэффициентов совместно равны нулю в генеральной совокупности.

$$
H_0:\gamma_{0}=\cdots=\gamma_{k}=0
$$

__Интерпретация__: включённые в модель новые регрессоры совместно не влияют на зависимую переменную (в генеральной совокупности)

__Тестовая статистика__ для выбранной оценки ковариационной матрицы $\hat{V}$ формула для F-статистики (всё вычисляется автоматически!)

$$
F=\frac{R^2_{unres}-R^2_{res}}{1-R^2_{unres}}\cdot \frac{n-k-1}{J}
$$

__Критическое значение__: $F$-распределения со степенями свободы ($k$ - число объясняющих переменных в модели (2))

$$ 
\begin{aligned} 
	df_1&=dfn=J & df_2&=dfd=n-k-1
\end{aligned}
$$

__Замечание__: 
* для F-тест есть неробастный (по умолчанию) и робастный вариант
* для подогнанной __регрессии со структурными изменениями__ используем метод `.f_test(r_matrix)` или `.wald_test(r_matrix, use_f=True)`
* тестируемую гипотезу специфицируем в виде `'x1=x2=0'` или `'x1=0,x2=0'` (параметр `r_matrix`) 

__Вывод__

**<span style="color:purple">Способ 1:</span>** используем $F$-статистику

* Отвергаем гипотезу $H_0$ при $F>F_{cr}$, <i><b><span style="color: purple">структурные изменения значимы</span></b><br>
* Не отвергаем гипотезу $H_0$ при $F<F_{cr}$, <i><b><span style="color: purple">структурные изменения незначимы</span> </b><br>

**<span style="color:purple">Способ 2:</span>** используем $P-value$

* Отвергаем гипотезу $H_0$  при $P<\alpha$, <i><b><span style="color: purple">структурные изменения значимы</span> </b><br>
* Не отвергаем гипотезу $H_0$ при $P>\alpha$, <i><b><span style="color: purple">структурные изменения незначимы</span> </b><br>

</div>

Для набора данных `sleep75` рассмотрим линейную регрессию __sleep на totwrk, age, smsa__: исходную и со структурными сдвигами относительно гендерной dummy-переменной __male__

Тестируем значимость структурных сдвигов (т.е. тестируем совместную значимость __male, totwrk:male, age:male, smsa:male__ для регрессии со структурными изменениями)

$$
	H_0:\beta_{male}=\beta_{totwrk:male}=\beta_{age:male}=\beta_{smsa:male}=0
$$

Для этой гипотезы $J=4$

In [ ]:
# импорт данных
df = pd.read_csv('sleep75.csv')

## F-тест (неробастный)

Используем OLS-оценку ковариационной матрицы $\hat{V}=s^2(X^\top X)^{-1}$

In [ ]:
# спецификация исходной модели 
mod = ols(formula='sleep~1+totwrk+age+smsa', data = df)
# подгонка исходной модели
res = mod.fit()
# спецификация модели со структурными сдвигами
mod_breaks = ols(formula='sleep~1+totwrk+age+smsa+male+totwrk:male+age:male+smsa:male', data = df)
# подгонка модели со структурными сдвигами
res_breaks = mod_breaks.fit()
# Результаты двух оцениваний в одной таблице
summary_col([res, res_breaks], stars=True, 
            regressor_order=['totwrk', 'age', 'smsa', 'male', 'totwrk:male', 'age:male', 'smsa:male'])

In [ ]:
F_test = res_breaks.f_test('male=0, totwrk:male=0, age:male=0, smsa:male=0')
print(F_test)

In [ ]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

In [ ]:
# степени свободы 
F_test.df_num, F_test.df_denom

In [ ]:
# альтернативно
W_test = res_breaks.wald_test('male=0, totwrk:male=0, age:male=0, smsa:male=0', use_f=True)
print(W_test)

## Критическое значение F-распределения

In [ ]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.isf(q=sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">структурные изменения незначимы</span></b><br>

</div>

## F-тест (робастный)

Используем HC3-оценку ковариационной матрицы

In [ ]:
# спецификация исходной модели 
mod = ols(formula='sleep~1+totwrk+age+smsa', data = df)
# подгонка исходной модели с робастной HC3 ковариационной матрицей
res_hc = mod.fit(cov_type='HC3')
# спецификация модели со структурными сдвигами
mod_breaks = ols(formula='sleep~1+totwrk+age+smsa+male+totwrk:male+age:male+smsa:male', data = df)
# подгонка модели со структурными сдвигами с робастной HC3 ковариационной матрицей
res_breaks_hc = mod_breaks.fit(cov_type='HC3')
# Результаты двух оцениваний в одной таблице
summary_col([res_hc, res_breaks_hc], stars=True, 
            regressor_order=['totwrk', 'age', 'smsa', 'male', 'totwrk:male', 'age:male', 'smsa:male'])

In [ ]:
F_test = res_breaks_hc.f_test('male=0, totwrk:male=0, age:male=0, smsa:male=0')
print(F_test)

In [ ]:
# Тестовая статистика и её P-значение
F_test.statistic, F_test.pvalue

In [ ]:
# альтернативно
W_test = res_breaks_hc.wald_test('male=0, totwrk:male=0, age:male=0, smsa:male=0', use_f=True)
print(W_test)

## Критическое значение F-распределения

In [ ]:
# уровень значимости
sign_level = 0.05
# Критическое значение F-распределения
f.isf(q=sign_level, dfn=F_test.df_num, dfd=F_test.df_denom)

## Вывод

<div style="background-color:Bisque; color:DarkBlue; padding:30px;">

Не отвергаем гипотезу $H_0$ (т.к. $F<F_{cr}$ или $P>\alpha$), <i><b><span style="color: purple">структурные изменения незначимы</span></b><br>

</div>